In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
folder = '../../data/data1'
items_set=pickle.load(open(folder + '/items_index_set','rb'))
bundle_item_map=pickle.load(open(folder + '/bundle_item_map','rb'))
user_bundle_map=pickle.load(open(folder + '/user_bundle_map','rb'))
user_item_map=pickle.load(open(folder + '/user_item_map','rb'))
item_data=pickle.load(open(folder + '/all_item_data','rb'))
itemID_appID_map = pickle.load(open(folder + '/item_id_lookup','rb'))
item_id_lookup = pickle.load(open(folder + '/item_id_lookup','rb'))

In [15]:
for itemID in range(len(itemID_appID_map)):
    appID = itemID_appID_map[itemID]
    print appID
    break

326950


In [20]:
appID_item_map = {}
for item in item_data:
    appID = item['item_id']
    appID_item_map[appID] = item

In [22]:
itemID_item_map = {}
for itemID in range(len(itemID_appID_map)):
    appID = itemID_appID_map[itemID]
    item = appID_item_map[appID]
    itemID_item_map[itemID] = item

In [23]:
with open(folder + '/itemID_item_map', 'wb') as handle:
    pickle.dump(itemID_item_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(folder + '/appID_item_map', 'wb') as handle:
    pickle.dump(appID_item_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
buy_items = set([])
for user in user_item_map:
    for itemID in user_item_map[user]:
        buy_items.add(itemID)

In [26]:
for itemID in buy_items:
    item = itemID_item_map[itemID]

In [28]:
def convert2Float(x):
    if isinstance(x, float):
        return -1
    else:
        idx = x.find('$')
        if idx == -1:
            return 0
        else:
            return float((x[idx+1:]))

In [34]:
itemID_price_map = {}
itemID_discount_map = {}
orign, discount, strange = 0, 0, 0
for itemID in itemID_item_map:
    item = itemID_item_map[itemID]
    if 'discount_final_price' not in item:
        itemID_price_map[itemID] = -2
        itemID_discount_map[itemID] = 0
    elif item['discount_final_price'] == '' and item['original_price'] == '':
        itemID_price_map[itemID] = -3
        itemID_discount_map[itemID] = 0
    elif item['original_price'] == '' and item['discount_original_price'] != '' and\
        item['discount_final_price'] != '':
        p2 = convert2Float(item['discount_original_price'])
        p3 = convert2Float(item['discount_final_price'])
        p1 = p2 + p3
        itemID_discount_map[itemID] =  1 - (float(p3) / p1)
    elif item['discount_final_price'] == '' or item['original_price'] == '':
        if item['discount_final_price'] == '':
            itemID_price_map[itemID] = convert2Float(item['original_price'])
            orign += 1
        else:
            itemID_price_map[itemID] = convert2Float(item['discount_final_price'])
            discount += 1
        itemID_discount_map[itemID] = 0
    else:
        p3 = convert2Float(item['discount_final_price'])
        p1 = convert2Float(item['original_price'])
        itemID_price_map[itemID] = int(p1 + p3 / 2)
        if p3 < p1:
            strange += 1
            itemID_discount_map[itemID] =   1 - (float(p3) / p1)
        else:
            itemID_discount_map[itemID] = 0

In [36]:
print orign, discount, strange

17 4 49


In [37]:
with open(folder + '/itemID_discount_map', 'wb') as handle:
    pickle.dump(itemID_discount_map, handle, protocol=pickle.HIGHEST_PROTOCOL)